In [1]:
# pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

##Load Dataset

In [4]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
type(squad)

datasets.arrow_dataset.Dataset

In [6]:
squad = squad.train_test_split(test_size=0.2)

##Preprocessing

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training

In [10]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [11]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [12]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [13]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [14]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [36]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="distilbert-QA-squad",
    tokenizer=tokenizer,
)

/content/distilbert-QA-squad is already a clone of https://huggingface.co/ashishverma20/distilbert-QA-squad. Make sure you pull the latest changes with `repo.git_pull()`.


In [35]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=[callback])

Epoch 1/5
250/250 [==============================] - 188s 751ms/step - loss: 1.6134 - val_loss: 1.8176
Epoch 2/5
250/250 [==============================] - 186s 745ms/step - loss: 1.6140 - val_loss: 1.8176
Epoch 3/5
250/250 [==============================] - 183s 732ms/step - loss: 1.6257 - val_loss: 1.8176
Epoch 4/5
250/250 [==============================] - 185s 738ms/step - loss: 1.6175 - val_loss: 1.8176
Epoch 5/5
250/250 [==============================] - 182s 728ms/step - loss: 1.6145 - val_loss: 1.8176


In [37]:
model.save('/content/drive/MyDrive/Fintech_news_sentiment/distilbert-QA5-squad.keras')

## Inference

In [38]:
question = "Which global center does Paris reflect?"
context = "Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine"

'art, fashion, gastronomy and culture'

In [39]:
inputs = tokenizer(question, context, return_tensors="tf")
outputs = model(**inputs)

In [40]:
outputs

TFQuestionAnsweringModelOutput(loss=None, start_logits=<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
array([[-2.4472096 , -2.8752277 , -3.4782412 , -3.9052608 , -3.5521364 ,
        -4.0567327 , -3.4675522 , -3.9222522 , -4.3406763 ,  0.62780577,
        -3.5708199 , -1.5133266 , -3.6817234 , -3.6357095 , -2.6458967 ,
        -4.720142  , -3.2386494 , -1.9981278 , -2.1209583 , -0.77579725,
        -2.631045  , -4.00992   , -1.8838995 , -1.3351864 , -2.9245837 ,
        -3.826032  ,  2.6525965 , -4.2153244 ,  2.0273104 , -4.1488132 ,
         1.9318857 , -2.2540414 , -2.632922  , -3.3758276 ,  0.4130237 ,
        -4.3661904 , -3.1020925 , -1.4026368 , -4.0906773 , -3.994998  ,
        -2.230548  , -3.7642977 , -3.7529757 , -3.861569  , -3.2233741 ,
        -3.7972193 , -3.8687406 , -4.120598  , -3.7070858 , -0.60803735,
        -2.1907594 , -4.251029  , -4.0577292 , -1.5058144 , -0.32538792,
        -2.2211225 , -4.3407974 ]], dtype=float32)>, end_logits=<tf.Tensor: shape=(1, 57), dt

In [41]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
answer_start_index, answer_end_index

(26, 34)

In [42]:

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'art, fashion, gastronomy and culture'

In [45]:
def answer(question, context):
    inputs =tokenizer(question,context,return_tensors="tf")
    outputs = model(**inputs)
    answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
    answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens)

answer(question,context)